# SIMPLE NEURAL NETWORK
* Will be using the UC Irvine Iris Dataset
* Data can be obtained from https://archive.ics.uci.edu/dataset/53/iris

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Create a Model Class that inherits nn.Module
class Model(nn.Module):
    # Input layer (4 heatures of the flower) --> 
    # Hidden Layer1 (number of neurons) -->
    # H2 (n) -->
    # output (3 classes of Iris flowers)
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__() # Instantiate our nn.Module
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x

In [3]:
torch.manual_seed(41)

In [4]:
# Create an instance of Model
model = Model()